In [2]:
%load_ext autoreload
%autoreload 2
from mail_agent.app import init_app, MailAgentApp
from mail_agent.models import User
from mail_agent.gmail import Gmail




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
app = init_app()

# Check if user needs re-authentication
with app.app_context():
    
    first_user = User.query.first()

    for email in first_user.emails:
        print(email)

<Email 1970d127156cffe1 from="Medium Daily Digest <noreply@medium.com>" subject="You Can Make Money With AI Without Quitting Your J...">
<Email 1970d0a8e834a59e from="Eric Busboom <eric.busboom@jointheleague.org>" subject="Re: Need my League email password reset...">
<Email 1970ce7baa36d3fc from="The Social Coyote <thesocialcoyote@mail.beehiiv.com>" subject="🐺 12 happenings and a note about event locations...">
<Email 1970cac163aa9f2f from=""Anthropic, PBC" <failed-payments@mail.anthropic.com>" subject="$20.00 payment to Anthropic, PBC was unsuccessful ...">
<Email 1970c81e5e108005 from="leaguebot@jointheleague.org" subject="Cancellations...">
<Email 1970c81e06d386fb from="leaguebot@jointheleague.org" subject="Cancellations...">
<Email 1970c0e38a3d6be0 from="Udacity <hello@udacity.com>" subject="⏰ Ends Today: 50% Off for Memorial Day...">
<Email 1970bcf62742d199 from="leaguebot@jointheleague.org" subject="[Receipt] Make-Up Lab Cencellation Reminder Emails...">
<Email 1970ba74050e33a8 f